In [1]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [8]:
import json
import numpy as np
#import tensorflow.keras.preprocessing.sequence
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
#import keras.preprocessing.sequence.pad_sequences


In [25]:
dataset_path = "C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\PreprocessDataset"

In [26]:
def find_image_files(folder_path):
    image_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith((".jpg", ".jpeg", ".png")):
                image_files.append(os.path.join(root, file))
               
    return image_files

# Find all image files in the dataset path
image_files = find_image_files(dataset_path)

In [27]:
image_files

['C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\PreprocessDataset\\video194_filtered\\frame_0000.jpg',
 'C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\PreprocessDataset\\video194_filtered\\frame_0001.jpg',
 'C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\PreprocessDataset\\video194_filtered\\frame_0002.jpg',
 'C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\PreprocessDataset\\video194_filtered\\frame_0003.jpg',
 'C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\PreprocessDataset\\video194_filtered\\frame_0004.jpg',
 'C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\PreprocessDataset\\video194_filtered\\frame_0005.jpg',
 'C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\PreprocessDataset\\video194_filtered\\frame_0006.jpg',
 'C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\PreprocessDataset\\video194_filtered\\frame_0007.jpg',
 'C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\PreprocessDataset\\video194_filtered\\frame_0008.jpg',
 

In [38]:
# Load JSON file containing captions
with open('C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\image_frames_full.json', 'r') as f:
    captions = json.load(f)

# Extract text and frames from captions
captions_text = [caption['caption'] for caption in captions]
captions_frame_paths = [caption['folder_path'] for caption in captions]

# Tokenize captions text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions_text)
captions_sequences = tokenizer.texts_to_sequences(captions_text)


In [39]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [40]:


# Tokenize captions text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions_text)
captions_sequences = tokenizer.texts_to_sequences(captions_text)

# Pad sequences to a fixed length
max_sequence_length = max(len(seq) for seq in captions_sequences)
padded_captions_sequences = pad_sequences(captions_sequences, maxlen=max_sequence_length)




In [41]:

# Define LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(units=128))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='linear'))

In [42]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(padded_captions_sequences, np.zeros(len(captions)), epochs=5, batch_size=32)

Epoch 1/5
575/575 [==============================] - 13s 17ms/step - loss: 2.5747e-06
Epoch 2/5
575/575 [==============================] - 10s 17ms/step - loss: 8.9395e-08
Epoch 3/5
575/575 [==============================] - 10s 17ms/step - loss: 4.0707e-08
Epoch 4/5
575/575 [==============================] - 10s 17ms/step - loss: 3.7956e-08
Epoch 5/5
575/575 [==============================] - 10s 17ms/step - loss: 2.2650e-08


In [43]:
# Define a query
query = "a man in a band playing a guitar"

# Tokenize and pad the query
query_sequence = tokenizer.texts_to_sequences([query])
padded_query_sequence = pad_sequences(query_sequence, maxlen=max_sequence_length)

In [44]:
import os

In [45]:
import numpy as np
from scipy.spatial.distance import cosine

# Assuming you have two pad sequences: seq1 and seq2
seq1 = padded_query_sequence
seq2_list = padded_captions_sequences

# Convert the sequences to numpy arrays
seq1 = np.array(seq1)
seq2_list = np.array(seq2_list)

# Compute cosine similarity between seq1 and each seq2 in seq2_list
similarity_scores = [1 - cosine(seq1, seq2) for seq2 in seq2_list]

# Define a threshold to filter out low similarity scores
similarity_threshold = 0.8

# Retrieve the indices of frames with similarity scores above the threshold
relevant_indices = [i for i, score in enumerate(similarity_scores) if score > similarity_threshold]

# Retrieve the corresponding frames from relevant_indices
relevant_frames = [image_files[i] for i in relevant_indices]

# Print the relevant frames
for frame in relevant_frames:
    print(frame)


ValueError: Input vector should be 1-D.

In [46]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Assuming you have two pad sequences: seq1 and seq2
seq1 = padded_query_sequence
seq2_list = padded_captions_sequences

# Reshape seq1 and seq2_list to 1-D arrays
seq1 = np.reshape(seq1, (seq1.shape[1],))
seq2_list = np.transpose(seq2_list)

# Compute cosine similarity between seq1 and each seq2 in seq2_list
similarity_scores = cosine_similarity([seq1], seq2_list)[0]

# Define a threshold to filter out low similarity scores
similarity_threshold = 0.8

# Retrieve the indices of frames with similarity scores above the threshold
relevant_indices = [i for i, score in enumerate(similarity_scores) if score > similarity_threshold]

# Retrieve the corresponding frames from relevant_indices
relevant_frames = [image_files[i] for i in relevant_indices]

# Print the relevant frames
for frame in relevant_frames:
    print(frame)


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 15 while Y.shape[1] == 18391

In [48]:
# Use the model to predict the caption with the highest similarity score
similarity_scores = model.predict(padded_captions_sequences)
max_index = np.argmax(similarity_scores)

start_index = max_index + 1
end_index = min(start_index + 20, len(captions))
relevant_frame_paths = [frame_path for frames in image_files[start_index:end_index] for frame_path in frames]

# Construct file paths for relevant frames
frame_dir = 'C:\\Users\\admin\\Desktop\\SCET_Major\frmes to json\PreprocessDataset'
relevant_frame_file_paths = [os.path.join(frame_dir, frame_path) for frame_path in relevant_frame_paths]


575/575 [==============================] - 1s 3ms/step


In [49]:
import json

# Specify the path of the JSON file\
json_file_path = 'C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\image_frames_full.json'

# Create a dictionary to store the relevant frame file paths
data = {'relevant_frames': relevant_frame_file_paths}

# Write the data to the JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file)


In [50]:
import os

In [51]:
print(relevant_frame_file_paths)


['C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\C', 'C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\:', 'C:\\', 'C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\U', 'C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\s', 'C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\e', 'C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\r', 'C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\s', 'C:\\', 'C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\a', 'C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\d', 'C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\m', 'C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\i', 'C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\n', 'C:\\', 'C:\\Users\\admin\\Desktop\

In [24]:
import os
import shutil
import json

# Specify the directory to store the relevant frames
output_dir = 'C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\matching_frames'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Iterate over the relevant frame file paths
for i, frame_file_path in enumerate(relevant_frame_file_paths):
    # Create a new file name using the index and the original file name
    file_name = f'relevant_frame_{i}.jpg'
    
    # Specify the output file path by joining the output directory and the new file name
    output_file_path = os.path.join(output_dir, file_name)
    
    # Copy the frame file to the output directory
    shutil.copy(frame_file_path, output_file_path)

    # Print the output file path
    print(f'Saved relevant frame {i+1}: {output_file_path}')

# Specify the path of the JSON file
json_file_path = 'C:\\Users\\admin\\Desktop\\SCET_Major\\frmes to json\\image_frames_full.json'

# Create a dictionary to store the relevant frame file paths
data = {'relevant_frames': relevant_frame_file_paths}

# Write the data to the JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file)


OSError: [Errno 22] Invalid argument: 'C:\\Users\\admin\\Desktop\\SCET_Major\x0crmes to json\\PreprocessDataset\\C'

In [18]:
import cv2
import os

def convert_frames_to_video(input_list, output_file_name, fps, size):
    # Define the output video writer object 
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Define the codec (FourCC code)
    out = cv2.VideoWriter(output_file_name, fourcc, fps, size)
    
    # Loop through the input frames and add them to the output video
    for i, img_name in enumerate(input_list):
        img_path = os.path.join(input_frame_path, img_name)
        img = cv2.imread(img_path)
        
        if img is None: # Skip if the image cannot be read
            print(img_name + ' does not exist')
            continue
        
        out.write(img) # Write out frame to video
        
        # Show the image in a window (optional)
        cv2.imshow('img', img)
        cv2.waitKey(1)
    
    # Release everything if job is finished
    out.release()
    cv2.destroyAllWindows()
    
    print("The output video {} is saved.".format(output_file_name))

if __name__=='__main__':
    # Set up paths and directories
    data_dir = 'D:\\Major23'
    data_subdir = 'Filtered'
    input_frame_path = os.path.join(data_dir, data_subdir)
    
    output_vid_dir = 'D:\\Major23\\video'
    if not os.path.exists(output_vid_dir):
        os.mkdir(output_vid_dir)
    
    # Set up parameters for output video
    fps = 5
    output_file_name = os.path.join(output_vid_dir, 'outputvideo1.mp4')
    
    # Get the size of the frames from the first image
    frame = cv2.imread(os.path.join(input_frame_path, os.listdir(input_frame_path)[0]))
    height, width, channels = frame.shape
    size = (width, height)
    
    # Convert the frames to a video
    img_list = sorted(os.listdir(input_frame_path)) # Sort the image names
    convert_frames_to_video(img_list, output_file_name, fps, size)


The output video D:\Major23\video\outputvideo21.mp4 is saved.


In [21]:
import cv2
import os

# Set up video writer
video_dir = 'D:\\Major23\\video'
if not os.path.exists(video_dir):
    os.makedirs(video_dir)
video_path = os.path.join(video_dir, 'output_video.mp4')
frame_width = 640  # Adjust to desired frame size
frame_height = 480
fps = 30
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(video_path, fourcc, fps, (frame_width, frame_height))

# Iterate over frames and add to video
frame_dir = 'D:\\Major23\\Filtered'  # Specify the directory where the frames are located
for frame_path in relevant_frame_file_paths:
    # Load frame
    frame = cv2.imread(frame_path)

    # Check if frame is valid
    if frame is not None:
        # Resize frame to desired dimensions
        frame = cv2.resize(frame, (frame_width, frame_height))

        # Write frame to video
        video_writer.write(frame)

# Release video writer
video_writer.release()
